# Fine-Tuning 

In [13]:
# Завантаження даних з txt з вказаним кодуванням utf-8
with open('data/gpt_translate_2.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()

# Ініціалізуємо списки для текстів та емоджі
texts = []
labels = []

# Обробляємо кожен рядок
for line in data:
    # Перевіряємо, чи є розділення
    if ' ,' in line:
        text, emoji = line.split(' ,', 1)  # Розділяємо тільки перший раз
        texts.append(text.strip())  # Текст до емоджі
        labels.append(emoji.strip())  # Емоджі після коми

# Перевіримо перші кілька записів
print(texts[:5])
print(labels[:5])


["Mat's childhood took place in a little African village in south Botswana", 'Conduct of the Persian Gulf War: Final Report to the Congress', 'It is even more striking that both EFhd2 and tau exhibit F-actin-bundling activity', 'Obesity leads to insulin resistance', 'The menstrual period']
['where he was initiated to sacred dancings and musical habits of Congalaabawanaa.,🌍👦🏾🏝️🌍🎶🥁🕺🏾', 'Appendix C: Intelligence, April 1992. back to article 2.,🔍📚', 'that actin dynamics are altered in tauopathies and that EFhd2 was found in synapses [6,7].,😮\u200d💨🔝🚶\u200d♂️👀🔍🤍🦠🏼📏🔗🩺🏢🥱💫🛠🧬🔄🔬🍃😶\u200d🌠🐇🔝🎑🔓📘', 'high blood pressure and ultimately to heart disease.,❤️🔺💔', "a metaphor alluding to sexual-abstinence . ',🩸🔴😅🚫💑"]


In [14]:
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset

# Завантажуємо токенізатор
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Токенізація текстів
encoding = tokenizer(texts, truncation=True, padding=True, max_length=128)

# Перетворюємо емоджі в числові мітки
emoji_labels = list(set(labels))  # Унікальні емоджі
label_map = {emoji: idx for idx, emoji in enumerate(emoji_labels)}

# Заміна емоджі на їх числові мітки
labels_numeric = [label_map[label] for label in labels]

# Розділення на тренувальний і валідаційний набори
train_texts, val_texts, train_labels, val_labels = train_test_split(encoding['input_ids'], labels_numeric, test_size=0.2)

# Конвертуємо в Torch Tensor
train_inputs = torch.tensor(train_texts)
val_inputs = torch.tensor(val_texts)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Створення DataLoader для тренування
train_data = TensorDataset(train_inputs, train_labels)
val_data = TensorDataset(val_inputs, val_labels)

train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=16)


In [15]:
from transformers import BertForSequenceClassification
from torch.optim import AdamW

# Завантажуємо модель BERT для класифікації
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))

# Оптимізатор
optimizer = AdamW(model.parameters(), lr=1e-5)

# Функція тренування
def train(model, train_dataloader):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, labels = batch
        optimizer.zero_grad()

        # Прямий прохід
        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        total_loss += loss.item()
    print(f"Training loss: {total_loss / len(train_dataloader)}")

# Тренуємо модель
for epoch in range(3):  # Змінити кількість епох за необхідності
    print(f"Epoch {epoch + 1}")
    train(model, train_dataloader)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1
Training loss: 4.591485595703125
Epoch 2
Training loss: 4.586425018310547
Epoch 3
Training loss: 4.561312961578369


In [16]:
# Збереження моделі та токенізатора
model.save_pretrained('path_to_save_model')
tokenizer.save_pretrained('path_to_save_model')


('path_to_save_model\\tokenizer_config.json',
 'path_to_save_model\\special_tokens_map.json',
 'path_to_save_model\\vocab.txt',
 'path_to_save_model\\added_tokens.json')

In [17]:
# Завантажуємо збережену модель
model = BertForSequenceClassification.from_pretrained('path_to_save_model')
tokenizer = BertTokenizer.from_pretrained('path_to_save_model')

# Використання моделі для передбачень
inputs = tokenizer("Some text to predict", return_tensors='pt')
outputs = model(**inputs)

# Отримання результату
predictions = outputs.logits.argmax(dim=-1)
print(predictions)


tensor([44])


In [19]:
# Завантаження необхідних бібліотек
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForSequenceClassification, BertTokenizer
from torch.optim import AdamW
from sklearn.model_selection import train_test_split

# Завантаження даних з txt
with open('data/gpt_translate_2.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()

texts = []
labels = []

# Обробка даних
for line in data:
    if ' ,' in line:
        text, emoji = line.split(' ,', 1)  # Розділення
        texts.append(text.strip())
        labels.append(emoji.strip())

# Токенізація
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoding = tokenizer(texts, truncation=True, padding=True, max_length=128)

# Перетворення емоджі в числові мітки
emoji_labels = list(set(labels))  # Унікальні емоджі
label_map = {emoji: idx for idx, emoji in enumerate(emoji_labels)}
labels_numeric = [label_map[label] for label in labels]

# Розділення на тренувальний і валідаційний набори
train_texts, val_texts, train_labels, val_labels = train_test_split(encoding['input_ids'], labels_numeric, test_size=0.2)

# Перетворення в Torch Tensor
train_inputs = torch.tensor(train_texts)
val_inputs = torch.tensor(val_texts)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Створення DataLoader
train_data = TensorDataset(train_inputs, train_labels)
val_data = TensorDataset(val_inputs, val_labels)

train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=16)

# Завантаження моделі
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))

# Оптимізатор
optimizer = AdamW(model.parameters(), lr=1e-5)

# Функція для тренування з валідацією
def train(model, train_dataloader, val_dataloader, optimizer, epochs=3):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        for batch in train_dataloader:
            input_ids, labels = batch
            input_ids, labels = input_ids.to(device), labels.to(device)
            optimizer.zero_grad()

            # Прямий прохід
            outputs = model(input_ids, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1} - Training loss: {avg_train_loss}")

        # Оцінка на валідаційному наборі
        model.eval()
        total_val_loss = 0
        correct_preds = 0
        total_preds = 0
        with torch.no_grad():
            for batch in val_dataloader:
                input_ids, labels = batch
                input_ids, labels = input_ids.to(device), labels.to(device)

                outputs = model(input_ids, labels=labels)
                loss = outputs.loss
                total_val_loss += loss.item()

                # Оцінка точності
                preds = outputs.logits.argmax(dim=-1)
                correct_preds += (preds == labels).sum().item()
                total_preds += labels.size(0)

        avg_val_loss = total_val_loss / len(val_dataloader)
        accuracy = correct_preds / total_preds
        print(f"Epoch {epoch + 1} - Validation loss: {avg_val_loss} - Accuracy: {accuracy}")

# Тренуємо модель
train(model, train_dataloader, val_dataloader, optimizer, epochs=3)

# Збереження моделі та токенізатора
model.save_pretrained('path_to_save_model')
tokenizer.save_pretrained('path_to_save_model')

# Завантажуємо збережену модель для подальших передбачень
model = BertForSequenceClassification.from_pretrained('path_to_save_model')
tokenizer = BertTokenizer.from_pretrained('path_to_save_model')

# Використання моделі для передбачень
inputs = tokenizer("Some text to predict", return_tensors='pt')
outputs = model(**inputs)

# Отримання результату
predictions = outputs.logits.argmax(dim=-1)
print(predictions)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 - Training loss: 4.600806045532226
Epoch 1 - Validation loss: 4.570686340332031 - Accuracy: 0.0
Epoch 2 - Training loss: 4.553167629241943
Epoch 2 - Validation loss: 4.659895658493042 - Accuracy: 0.0
Epoch 3 - Training loss: 4.5578405380249025
Epoch 3 - Validation loss: 4.667123079299927 - Accuracy: 0.0


SafetensorError: Error while serializing: IoError(Os { code: 1224, kind: Uncategorized, message: "The requested operation cannot be performed on a file with a user-mapped section open." })